# Create and run Earthquake Estimator

In [1]:
!nvidia-smi

Tue Apr 16 01:09:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
import densenet
from importlib import reload
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
import pickle
import utils
%matplotlib inline

In [3]:
def reload_tf():
    global utils
    for i in range(2):
        import utils
        import freq_preprocess
        import stat_preprocess
        reload(utils)
        reload(freq_preprocess)
        reload(stat_preprocess)
        del utils
        del freq_preprocess
        del stat_preprocess
    
    global lstm_estimator
    global earthquake_input_fn
    global densenet
    global hooks
    global lstm
    global utils
    global freq_preprocess
    global stat_preprocess
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        import hooks
        import densenet
        import lstm
        import utils
        import freq_preprocess
        import stat_preprocess
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        reload(hooks)
        reload(densenet)
        reload(lstm)
        reload(utils)
        reload(freq_preprocess)
        reload(stat_preprocess)
        del lstm_estimator
        del earthquake_input_fn
        del hooks
        del densenet
        del lstm
        del utils
        del freq_preprocess
        del stat_preprocess
    import lstm_estimator
    import earthquake_input_fn
    import densenet
    import hooks
    import lstm
    import utils
    import freq_preprocess
    import stat_preprocess
reload_tf()

<>:19: SyntaxWarning: name 'utils' is assigned to before global declaration
<>:20: SyntaxWarning: name 'freq_preprocess' is assigned to before global declaration
<>:21: SyntaxWarning: name 'stat_preprocess' is assigned to before global declaration
<>:19: SyntaxWarning: name 'utils' is assigned to before global declaration
<>:20: SyntaxWarning: name 'freq_preprocess' is assigned to before global declaration
<>:21: SyntaxWarning: name 'stat_preprocess' is assigned to before global declaration
<>:19: SyntaxWarning: name 'utils' is assigned to before global declaration
<>:20: SyntaxWarning: name 'freq_preprocess' is assigned to before global declaration
<>:21: SyntaxWarning: name 'stat_preprocess' is assigned to before global declaration
<ipython-input-3-e2710c6b8d00>:19: SyntaxWarning: name 'utils' is assigned to before global declaration
  global utils
<ipython-input-3-e2710c6b8d00>:20: SyntaxWarning: name 'freq_preprocess' is assigned to before global declaration
  global freq_preproces

In [4]:
def create_densenet_from_params(batch_size, timesteps, time_pool, growth_rate, layer_sizes, 
                                layer_dilations, layer_kernel_sizes, dropout_rate, compression_theta,
                                dense_size, feature_columns, optimizer_name, learning_rate, weight_decay,
                                momentum, nesterov, lambda_l2_reg, grad_clip, dtype,
                                use_lstm, lstm_layers, lstm_units, lstm_dropout,
                                model_dir=None):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'time_pool': time_pool,
        'growth_rate': growth_rate,
        'layer_sizes': layer_sizes,
        'layer_dilations': layer_dilations,
        'layer_kernel_sizes': layer_kernel_sizes,
        'dropout_rate': dropout_rate,
        'compression_theta': compression_theta,
        'dense_size': dense_size,
        'feature_columns': feature_columns,
        'optimizer_name': optimizer_name,
        'learning_rate': learning_rate,
        'weight_decay': weight_decay,
        'momentum': momentum,
        'nesterov': nesterov,
        'lambda_l2_reg': lambda_l2_reg,
        'grad_clip': grad_clip,
        'dtype': dtype,
        'use_lstm': use_lstm,
        'lstm_layers': lstm_layers,
        'lstm_units': lstm_units,
        'lstm_dropout': lstm_dropout
    }
    
    if model_dir is None:
        model_dir = '/workspace/persistent-data/models/densenet-%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
    
    params_file = osp.join(osp.split(model_dir)[0], osp.split(model_dir)[1] + '.params.pickle')
    if not osp.isfile(params_file):
        print('writing params to %s' % params_file)
        with open(params_file, 'wb') as f:
            pickle.dump(params, f)
        
    config = tf.estimator.RunConfig(model_dir=model_dir,
                                    log_step_count_steps=int(500 / batch_size),
                                    save_checkpoints_secs=300,
                                    session_config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.75)))
    
    estim = tf.estimator.Estimator(model_fn=densenet.densenet_model_fn,
                                   params=params,
                                   model_dir=model_dir,
                                   config=config)
    return estim

In [5]:
def train_and_evaluate_densenet(batch_size, timesteps, time_pool, growth_rate, layer_sizes, 
                                layer_dilations, layer_kernel_sizes, dropout_rate, compression_theta,
                                dense_size, feature_columns, optimizer_name, learning_rate, weight_decay, momentum, nesterov,
                                lambda_l2_reg, grad_clip,
                                noise,
                                earthquake_data_dir,
                                eval_count,
                                eval_every_n_secs,
                                epochs=1,
                                dtype='float64',
                                use_lstm=False, lstm_layers=0, lstm_units=0, lstm_dropout=0,
                                model_dir=None):
    
    estim = create_densenet_from_params(batch_size=batch_size, timesteps=timesteps, time_pool=time_pool, 
                                        growth_rate=growth_rate, layer_sizes=layer_sizes,
                                        layer_dilations=layer_dilations, layer_kernel_sizes=layer_kernel_sizes,
                                        dropout_rate=dropout_rate, compression_theta=compression_theta,
                                        dense_size=dense_size, feature_columns=feature_columns, 
                                        optimizer_name=optimizer_name, learning_rate=learning_rate,
                                        weight_decay=weight_decay, momentum=momentum, nesterov=nesterov,
                                        lambda_l2_reg=lambda_l2_reg, grad_clip=grad_clip, dtype=dtype,
                                        use_lstm=use_lstm, lstm_layers=lstm_layers, lstm_units=lstm_units,
                                        lstm_dropout=lstm_dropout,
                                        model_dir=model_dir)
    
    trainspec = tf.estimator.TrainSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                             batch_size,
                                                                             timesteps,
                                                                             noise=noise,
                                                                             window_shift=time_pool,
                                                                             traintest='train',
                                                                             epochs=epochs),
                                       max_steps=1000000)
    
    evalspec = tf.estimator.EvalSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                           batch_size,
                                                                           timesteps,
                                                                           noise=noise,
                                                                           window_shift=time_pool,
                                                                           traintest='test'),
                                     steps=eval_count,
                                     start_delay_secs=eval_every_n_secs, throttle_secs=eval_every_n_secs)
    
    return estim, trainspec, evalspec

In [28]:
reload_tf()

EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords5'
BATCH_SIZE = 64
TIMESTEPS = 150000
TIME_POOL = 75
GROWTH_RATE        = 4
LAYER_SIZES        = [ 8, 4]
LAYER_DILATIONS    = [ 2, 1]
LAYER_KERNEL_SIZES = [ 5, 3]
DROPOUT_RATE = 0.4
COMPRESSION_THETA = 0.5
DENSE_SIZE = 256
OPTIMIZER_NAME = 'MomentumW'
LEARNING_RATE = 0.01
WEIGHT_DECAY = 2e-4
MOMENTUM = 0.9
NESTEROV = True
LAMBDA_L2_REG = 0.0000  # weight decay instead
GRAD_CLIP = 1
NOISE = 0.1
EVAL_NUM_BATCHES = 200
EVAL_EVERY_N_SECONDS = 1800
EPOCHS = 20
DTYPE = 'float32'

USE_LSTM = True
LSTM_LAYERS = 1
LSTM_UNITS = 512
LSTM_DROPOUT = 0.4
FEATURE_COLUMNS = [tf.feature_column.numeric_column(key='acousticdata', dtype=tf.float32, shape=(int(TIMESTEPS/TIME_POOL)*\
                                                                                                 66))]  # stats + pcts + stft

estim, train_spec, eval_spec = train_and_evaluate_densenet(BATCH_SIZE, TIMESTEPS, TIME_POOL, GROWTH_RATE,
                                                           LAYER_SIZES,
                                                           LAYER_DILATIONS, LAYER_KERNEL_SIZES,
                                                           DROPOUT_RATE, COMPRESSION_THETA, DENSE_SIZE,
                                                           FEATURE_COLUMNS,
                                                           OPTIMIZER_NAME, LEARNING_RATE, WEIGHT_DECAY,
                                                           MOMENTUM,
                                                           NESTEROV, LAMBDA_L2_REG, GRAD_CLIP,
                                                           NOISE,
                                                           EARTHQUAKE_DATA_DIR,
                                                           EVAL_NUM_BATCHES,
                                                           EVAL_EVERY_N_SECONDS,
                                                           EPOCHS, DTYPE,
                                                           USE_LSTM, LSTM_LAYERS, LSTM_UNITS, LSTM_DROPOUT,
                                                           # MODEL_DIR
                                                          )

writing params to /workspace/persistent-data/models/densenet-2019-04-18-12-57-32.params.pickle
INFO:tensorflow:Using config: {'_service': None, '_experimental_distribute': None, '_train_distribute': None, '_task_id': 0, '_model_dir': '/workspace/persistent-data/models/densenet-2019-04-18-12-57-32', '_task_type': 'worker', '_eval_distribute': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.75
}
, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8dd0d37b38>, '_save_checkpoints_secs': 300, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_log_step_count_steps': 7, '_device_fn': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_tf_random_seed': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_protocol': None, '_num_worker_replicas': 1, '_master': ''}


In [ ]:
try:
    tf.estimator.train_and_evaluate(estim, train_spec, eval_spec)
except KeyboardInterrupt as e:
    raise e
except Exception as e:
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('--  TRAINING ERROR  --')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    raise e

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
stride_input_o1 Tensor("stack:0", shape=(2000, 75), dtype=float32, device=/device:CPU:0)
stride_input_o2 Tensor("stack_1:0", shape=(2000, 75), dtype=float32, device=/device:CPU:0)
stride_input_o3 Tensor("stack_2:0", shape=(2000, 75), dtype=float32, device=/device:CPU:0)
stride_input_o4 Tensor("stack_3:0", shape=(2000, 75), dtype=float32, device=/device:CPU:0)
stride_input_o5 Tensor("stack_4:0", shape=(2000, 75), dtype=float32, device=/device:CPU:0)
num_splits=2000 (this is the number of timesteps fed to stride CNN)
stride_input Tensor("stack_5:0", shape=(2000, 50), dtype=float32, device=/device:CPU:0)
acoustic_stat Tensor("stack_5:0", shape=(2000, 